In [46]:
import importlib

import rough_path_signatures_pricing.simulation_models as sm

importlib.reload(sm)

<module 'rough_path_signatures_pricing.simulation_models' from 'C:\\Users\\chris\\VSCodeProjects\\rough-path-signatures-pricing\\src\\rough_path_signatures_pricing\\simulation_models.py'>

In [59]:
params = {"x0": 1, "t0": 0, "t1": 1, "n_steps": 10, "n_paths": 5}
seed = 42

In [60]:
gbm = sm.GBM(mu=0, sigma=10, seed=seed)

In [49]:
gbm_paths = gbm.simulate_paths(**params)
gbm_paths

(array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
 array([[ 1.00000000e+00,  1.96360001e+00, -6.12221652e+00,
         -2.31474919e+01,  3.97516823e+01,  1.65134005e+01,
         -1.87499854e+00, -1.45733968e+01, -6.66022285e+01,
         -2.23142512e+02, -3.77457623e+02],
        [ 1.00000000e+00, -2.28871851e+00, -3.21397153e+00,
         -1.11190368e+01, -2.40848608e+01,  2.77767878e+01,
          7.45337171e+01, -2.12568020e+01, -1.35972641e+01,
         -3.69519877e+01, -1.38780571e+02],
        [ 1.00000000e+00,  3.37313505e+00, -1.58149308e-04,
         -1.91172054e-04,  3.88509986e-04,  1.89049548e-03,
          4.07521936e-03, -2.52603996e-03,  4.18516336e-03,
         -4.62262375e-03, -7.89114791e-03],
        [ 1.00000000e+00,  3.97432679e+00,  3.76317111e+00,
          1.71775578e+01,  6.48951603e+01,  3.31831584e+01,
          7.64929911e+01, -1.20352184e+02,  1.93434668e+02,
          3.35446386e+02,  1.05562066e+03],
        [ 1.00000000e+00, -5.16971499e+

In [61]:
def theta(t):
    return 0.03


# theta = lambda t: 0.03
hullwhite = sm.HullWhite(a=1, sigma=10, theta=theta, seed=seed)

In [64]:
hullwhite_paths = hullwhite.simulate_paths(**params)
hullwhite_paths

(array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
 array([[ 1.        ,  1.86660001, -2.43491315,  0.59247873, -2.1810905 ,
         -2.54456757, -3.40065487,  3.71489498,  6.91652934,  8.57825247,
          8.41498128],
        [ 1.        , -2.38571851, -1.7398798 ,  0.89670224,  1.97612438,
         -0.37177634,  1.35171074, -0.06565746, -0.41642522,  1.34582191,
          3.96993948],
        [ 1.        ,  3.27613505,  1.95147466,  1.9681346 , -1.25793189,
          2.73687646,  3.62182442,  1.64278824, -1.17529865, -3.15929526,
         -2.13329453],
        [ 1.        ,  3.87732679,  3.43946419,  6.66316745,  8.77775447,
          7.4143139 ,  7.98105762,  4.61257654,  1.54708036,  2.12953089,
          4.06649099],
        [ 1.        , -5.26671499, -7.43460525, -5.20975037, -4.84365493,
         -5.71078094, -3.77432721, -1.44599652,  0.75895817,  1.05505528,
          1.16625353]]),
 array([[ 0.09636   , -0.41178532,  0.27809006, -0.27173214, -0.05845861,
    